In [1]:
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql.functions import udf, from_utc_timestamp ,col, explode,from_json,regexp_replace,split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, TimestampType
from pyspark.sql import Row
from pyspark.sql.functions import date_format,to_timestamp, to_date,regexp_replace
import pyspark.sql.types as T
from pyspark.sql.functions import broadcast
spark.conf.set("spark.databricks.delta.autoCompact.enabled","true")
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")
spark.conf.set("spark.databricks.io.cache.enabled", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled","true")
spark.conf.set("spark.databricks.adaptive.autoOptimizeShuffle.enabled","true")

#spark.conf.set("spark.sql.shuffle.partitions", 64)

In [2]:
# from datetime import datetime
# import pytz
  
# # get the standard UTC time 
# UTC = pytz.utc
  
# # it will get the time zone 
# # of the specified location
# EST = pytz.timezone('US/Eastern')
  
# # print the date and time in
# # standard format
# print("UTC in Default Format : ", 
#       datetime.now(UTC))
  
# print("EST in Default Format : ", 
#       datetime.now(EST))
  
# # print the date and time in 
# # specified format
# datetime_utc = datetime.now(UTC)
# print("Date & Time in UTC : ",
#       datetime_utc.strftime('%Y:%m:%d %H:%M:%S %Z %z'))
  
# datetime_est = datetime.now(EST)
# print("Date & Time in eST : ", 
#       datetime_est.strftime('%Y:%m:%d %H:%M:%S %Z %z'))

In [3]:
%run /utils/merge_data_notebook

# 

# transform queue table

In [4]:
print(raw_adls_path+'pcs/queue/')
df_queue = spark.read.load(raw_adls_path+'pcs/queue/',format = "delta")
#display(df_queue)
df_queue_trans = df_queue.selectExpr('id as queueId', 
    'name as queueName', "split(name,'[.]')[0] as department"
    ,"split(name,'[.]')[1] as location"
    ,"split(name,'[.]')[2] as language_cd"
    ,"split(name,'[.]')[3] as brand"
    ,"split(name,'[.]')[4] as context","lastUpdateDate", "row_number() over(partition by id order by lastUpdateDate desc) as rn")\
    .where("rn=1")\
    .drop("rn").distinct()

#display(df_queue_trans)
#print(df_queue.count())


# get Agent DataFrame

In [5]:
df_agents = spark.read.load(raw_adls_path+'pcs/agents/',format = "delta")
df_na = spark.sql("select 'NA' as agent_id, 'NA' as agent_name, current_date as lastUpdateDate")
df_agents_trans = df_agents.selectExpr("id as agentId","name as agentName", "lastUpdateDate")
df_agents_trans = df_agents_trans.union(df_na).distinct()
df_agents_trans = df_agents_trans.selectExpr("agentId", "agentName", "lastUpdateDate", "row_number() over(partition by agentId order by lastUpdateDate desc) as rn" ).where("rn=1").drop("rn")
#df_agents_trans = df_agents_trans.where("rn=1").drop("rn")
df_lang = spark.sql(f"select language_short_nm as language_cd ,  language_long_nm as language from language_codes")
#display(df_agents_trans)

# perform transformation for PCS data

In [6]:
#spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df_pcs = spark.read.load(raw_adls_path+'pcs/pcs/',format = "delta")

#display(df_pcs)
# getdigit-b85fb4cc-2c33-4d52-83fb-6608ee944564: ""NA""
# waitTime: ""4318.0""
# holdDuration: ""0.0""
# getdigit-d930a6f4-f5d8-44d5-8729-c3ab0c53aba4: ""NA""
# callerPhoneNumber: ""7737169265""
# muteDuration: "0"
# timeInIVR: ""48342.0""
# callDate: ""02/22/2023 12:35:33""
# totalScore: "0"
# queueList: ""[219]""
# callerName: ""MEEKS BERNARD""
# agentList: ""[ag36_SFf8LTB6LfUxWqyG9Aw]""
# agentCallHandlingDuration: ""456302.0""
# transactionId: ""941019""
## to convert data types , use to_date, to_timestamp
## to have appropriate date formats MM/dd/yyyy use date_format
## from_unixtimestamp(unix_timestamp()) -- also can be used as alternative to date_format
## split('[.]' ) -- used to split a string with period 
df_pcs = df_pcs.selectExpr(
 "loadDate as loadDate"   
,"data.callId" 
,"to_date(data.callDate ,'MM/dd/yyyy') as callDate"
,"date_format(to_timestamp(data.callDate,'MM/dd/yyyy HH:mm:ss'),'HH:mm:ss') as callTimeEST"
,"from_unixtime(unix_timestamp(data.callDate,'MM/dd/yyyy HH:mm:ss'),'hh:mm:ss aa') as callTime12HrEST"
,"data.callDate as callDatetimeEST"
#,"from_utc_timestamp(to_timestamp(data.callDate,'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as CallDatetimeEST"
#,"date_format(from_utc_timestamp(to_timestamp(data.callDate,'MM/dd/yyyy HH:mm:ss'),'US/Eastern'),'hh:mm:ss aa') as CallTime12HrEST"
,"data.callerName"
,"data.callerPhoneNumber"
,"data.totalScore"
,"data.question1 as Q1"
,"data.question2 as Q2"
,"data.agentList as agentList"
,"data.queueList as queueList"
,"data.transactionId as transactionIdOriginal"
,"split(data.transactionId,' ')[0] as transactionId"
,"case when size(split(regexp_replace(data.agentList, '(^\[)|(\]$)', ''),','))>1 then 'NA' else split(regexp_replace(data.agentList, '(^\\\\[)|(\\\\]$)', ''),',')[0] end as agentId"
,"split(regexp_replace(data.queueList, '(^\\\\[)|(\\\\]$)', ''),',')[0] as queueId"
,"questionLabel"
,"data.agentCallHandlingDuration as agentCallDurationMs"
,"data.callDuration as callDurationMs"
,"data.holdDuration as holdDurationMs"
,"data.muteDuration as muteDurationMs"
,"data.timeInIVR as timeInIVRMs"
,"data.waitTime as waitTimeMs"
,"round(cast(data.agentCallHandlingDuration as float)/60000,2) as agentCallDuration"
,"round(cast(data.callDuration as float)/60000,2) as callDuration"
,"round(cast(data.holdDuration as float)/60000,2) as holdDuration"
,"round(cast(data.muteDuration as float)/60000,2) as muteDuration"
,"round(cast(data.timeInIVR as float)/60000,2) as timeInIVR"
,"round(cast(data.waitTime as float)/60000,2) as waitTime"
,"from_utc_timestamp(to_timestamp(loadDateTime,'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as loadDateTimeEST"
,"row_number() over(partition by data.callId order by loadDateTime desc) as rn"
)

df_pcs = df_pcs.join(broadcast(df_agents_trans),["agentId"], "left").drop("lastUpdateDate")
df_pcs = df_pcs.join(broadcast(df_queue_trans),["queueId"],"left").drop("lastUpdateDate")
df_pcs = df_pcs.join(broadcast(df_lang),["language_cd"],"left").select("*").where("rn = 1")
display(df_pcs.select('*'))
#df_pcs.explain()

# transform historical data

In [7]:
print(raw_adls_path+'historical_analytics/')
lst = mssparkutils.fs.ls(raw_adls_path+'historical_analytics/incremental/')#
folder_name = [j.name for j in lst]
folder_name = sorted(folder_name, reverse=True)[0]
print("latest folder name::",folder_name)
df = spark.read.load(raw_adls_path+'historical_analytics/incremental/'+folder_name,format = "delta")
spark.sql(f"optimize delta.`{raw_adls_path}historical_analytics/incremental/{folder_name}`")
#df.dtypes
#df = df.selectExpr('participants','queueWaitDuration').where("interactionId = 'int-18aafa907ce-TwSMHTchKftrlWWF9DQHK2zbZ-phone-03-wolverineworldwid01'")
#df = df.withColumn('queueWaitDuration',F.split(F.col('queueWaitDuration'),',')[0])\
 #           .withColumn('queueWaitDuration',F.split(F.col('queueWaitDuration'),':')[1])
#display(df)            

In [8]:
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql import Row
import pyspark.sql.functions as F

#df = spark.read.load(raw_adls_path+'historical_analytics/',format = "delta")
v_list = ['participantMute'
,'participantMuteDuration'
,'participantLongestMuteDuration'
,'scheduleHours'
,'TimetoAbandon'
,'Transfers'
,'interactionDuration'
,'ivrTreatmentDuration'
,'participantBusyDuration'
,'participantHandlingDuration'
,'participantHoldDuration'
,'participantLongestHoldDuration'
,'participantOfferDuration'
,'participantProcessingDuration'
,'participantWrapUpDuration'
,'queueWaitDuration'
]
for j in v_list:
    df = df.withColumn(j,F.split(F.col(j),',')[0])\
            .withColumn(j,F.split(F.col(j),':')[1])
df_hist=df.withColumn('finishedTime',F.split(F.col('finishedTime'),',')[0])\
     .withColumn('finishedTime',F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col('finishedTime'), '\\{', ''),"'value':",''),"'",''))             
   
participants_schema = StructType([
    StructField("participantAssignNumber", StringType(), True),
    #StructField("participantType", StringType(), True),
    StructField("participantId", StringType(), True),
    StructField("participantName", StringType(), True),
    StructField("participantOfferAction", StringType(), True),
    
])

df_hist = df_hist.selectExpr('loadDate', 
 'interactionId',
 'blindTransferToAgent',
 'blindTransferToQueue',
 #'campaignId',
 'channelId',
 #'creationTime as creationTimeUTC',
 "from_utc_timestamp(to_timestamp(date_format(creationTime,'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as creationTimeEST",
 'customerName',
 'destination',
 "regexp_replace(direction, '(.*)Dir', '$1') as direction", 
 'dispositionAction',
 'externalTransactionData',
 #'finishedTime as finishedTimeUTC',
 "from_utc_timestamp(to_timestamp(date_format(finishedTime,'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as finishedTimeEST",
 "regexp_replace(regexp_replace(interactionLabels, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as interactionLabels",
 'interactionType',
 'ivrTreatmentDuration/1000 as ivrTreatmentDuration ',
 #"unix_timestamp(ivrTreatmentDuration,'H:mm:ss') as ivrTreatmentDuration", #convert to seconds
 'mediaType',
 'originalInteractionId',
 'originalTransactionId',
 'origination',
 'outboundPhoneCode',
 'outboundPhoneCodeText',
 #'participantAssignNumber',
 'participantBusyDuration/1000 as participantBusyDuration',
 'participantHandlingDuration/1000 as participantHandlingDuration',
 #"unix_timestamp(participantBusyDuration,'H:mm:ss') as participantBusyDuration",
 #"unix_timestamp(participantHandlingDuration,'H:mm:ss') as participantHandlingDuration",
 "from_utc_timestamp(to_timestamp(date_format(participantHandlingEndTime, 'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as participantHandlingEndTime",
 'participantHold',
 'participantHoldDuration/1000 as participantHoldDuration',
 #"unix_timestamp(participantHoldDuration,'H:mm:ss') as participantHoldDuration", 
 #"regexp_replace(regexp_replace(participantId, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as agentId",
 #"unix_timestamp(participantLongestHoldDuration,'H:mm:ss') as participantLongestHoldDuration ",
'participantLongestHoldDuration/1000 as participantLongestHoldDuration',
 #"regexp_replace(regexp_replace(participantName, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as participantName",
 #"regexp_replace(regexp_replace(participantOfferAction, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as participantOfferAction",
 "from_utc_timestamp(to_timestamp(date_format(participantOfferActionTime, 'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as participantOfferActionTime",
 #"unix_timestamp(participantOfferDuration,'H:mm:ss') as participantOfferDuration",
 'participantOfferDuration/1000 as participantOfferDuration',
 "from_utc_timestamp(to_timestamp(date_format(participantOfferTime, 'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as participantOfferTime",
 'participantProcessingDuration/1000 as participantProcessingDuration',
 #"unix_timestamp(participantProcessingDuration, 'H:mm:ss') as participantProcessingDuration",
 "regexp_replace(regexp_replace(participantType, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as participantType",
 'participantWrapUpDuration/1000 as participantWrapUpDuration',
 #"unix_timestamp(participantWrapUpDuration,'H:mm:ss') as participantWrapUpDuration ",
 "from_utc_timestamp(to_timestamp(date_format(participantWrapUpEndTime, 'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as participantWrapUpEndTime",
 "split(regexp_replace(regexp_replace(queueId, '(^\\\\[)|(\\\\]$)', ''),'\\""'""',''),',')[0] as queueId",
 #"split(regexp_replace(regexp_replace(queueName, '(^\\\\[)|(\\\\]$)', ''),'\\""'""',''),',')[0] as queueName",
 "from_utc_timestamp(to_timestamp(date_format(queueTime,'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as queueTime",
 'queueWaitDuration/1000 as queueWaitDuration',
 #"unix_timestamp(queueWaitDuration,'H:mm:ss') as queueWaitDuration",
 "date_format(time,'yyyy-MM-dd') as callDate",
 #"from_utc_timestamp(to_timestamp(date_format(time,'MM/dd/yyyy HH:mm:ss'),'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as time",
 'transactionId',
 'warmTransfersCompleted',
 "regexp_replace(regexp_replace(wrapUpCode, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpCode",
 "regexp_replace(regexp_replace(wrapUpCodeId, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpCodeId",
 "regexp_replace(regexp_replace(wrapUpCodeList, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpCodeList",
 "regexp_replace(regexp_replace(wrapUpCodeListId, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpCodeListId",
 "regexp_replace(regexp_replace(wrapUpCodeText, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpCodeText",
 "regexp_replace(regexp_replace(wrapUpShortCode, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','') as wrapUpShortCode",
 "from_utc_timestamp(to_timestamp(loadDateTime,'MM/dd/yyyy HH:mm:ss'),'US/Eastern') as loadDateTimeEST",
 "row_number() over(partition by interactionId order by loadDateTime desc) as rn",
 "participantMute",
 "participantMuteDuration/1000 as participantMuteDuration",
 "participantLongestMuteDuration/1000 as participantLongestMuteDuration",
 "scheduleHours",
 "TimeToAbandon as timeToAbandon",
 "Transfers",
 "interactionDuration/1000 as interactionDuration",
 "participants"
)

df_hist = df_hist.withColumn("parsed_data", from_json(col("participants"), ArrayType(participants_schema)))
df_hist = df_hist.selectExpr("*", "cast(parsed_data.participantAssignNumber as string) as participantAssignNumber ","cast(parsed_data.participantId  as string) as agentId","cast(parsed_data.participantName as string) as participantName","cast(parsed_data.participantOfferAction as string) as participantOfferAction")
#df_hist = df_hist.join(df_agents_trans,["agentId"], "left")
df_hist = df_hist.drop("participants","parsed_data")
df_hist = df_hist.where("rn=1").join(broadcast(df_queue_trans),["queueId"],"left")
df_hist = df_hist.join(broadcast(df_lang),["language_cd"],"left").select("*")



display(df_hist)

In [9]:
#display(df.select(from_json(col("participants"),ArrayType(participants_schema))).where("interactionId = 'int-18aafa907ce-TwSMHTchKftrlWWF9DQHK2zbZ-phone-03-wolverineworldwid01'") )
#display(df_hist.selectExpr('loadDate','participantWrapUpDuration','queueWaitDuration','participantHandlingDuration','participantName').where("interactionId = 'int-18aafa907ce-TwSMHTchKftrlWWF9DQHK2zbZ-phone-03-wolverineworldwid01'"))

In [10]:


# refresh table  structured.cco_pcs;  
# select distinct callDate from structured.cco_historical_analytics order by 1 desc;
# select distinct callDate from structured.cco_pcs order by 1 desc;
# select distinct date_format(time,'yyyy-MM-dd') from  delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/historical_analytics` order by 1 desc;
# select distinct to_date(data.callDate ,'MM/dd/yyyy')  from  delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/pcs/pcs` order by 1 desc;         
# select 1 ; 


# Split InteractionLabels

In [11]:
#display(df_hist.selectExpr("regexp_replace(regexp_replace(interactionLabels, '(^\\\\[)|(\\\\]$)', ''),'\\""'""','')"))
#display(df_hist.selectExpr("split(regexp_replace(regexp_replace(interactionLabels, '(^\\\\[)|(\\\\]$)', ''),'\\""'""',''),',')[0]"))

## Optimize the table

In [12]:
#spark.sql(f"optimize delta.`{structured_adls_path}historical_analytics`")


# create a dict for source and target tables

In [13]:
dict_tables = {
      "pcs": { "source_df": df_pcs
                 ,"where_condition":"target.callId = source.callId"                        
                 ,"target_table":"lakedb_gold.cco_pcs" 
                 ,"partition":""
                 ,"path": gold_adls_path + 'cco/pcs'
                       
                   }
  ,"historical_analytics":
                     { 
                       "source_df": df_hist
                       ,"where_condition": "target.interactionId = source.interactionId"                     
                       ,"target_table":"lakedb_gold.cco_historical_analytics"
                       ,"partition":""
                       ,"path": gold_adls_path + 'cco/historical_analytics'
                     }  
  , "agent":
  
                    { 
                       "source_df": df_agents_trans
                       ,"where_condition": "target.agentId = source.agentId"                    
                       ,"target_table":"lakedb_gold.cco_agent"
                       ,"partition":""
                       ,"path": gold_adls_path + 'cco/agent'
                      
                    }
  , "queue":
  
                    { 
                       "source_df": df_queue_trans   
                       ,"where_condition": "target.queueId = source.queueId"                   
                       ,"target_table":"lakedb_gold.cco_queue"
                       ,"partition":""
                       ,"path": gold_adls_path + 'cco/queue'
                      
                    }                 
              }
              

#print(dict_tables)

In [14]:
perform_merge(dict_tables)